In [3]:
import glob
import os
import subprocess
import tarfile
import wget

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import nemo
import nemo.collections.asr as nemo_asr

# import fastwer

import json
import librosa

import librosa
import json
import copy

from omegaconf import DictConfig
import copy

import sys
import os
import pandas as pd
import subprocess
import json
import codecs
import unidecode
# import pitch

from tqdm import tqdm

[NeMo W 2022-03-27 10:17:39 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



In [10]:
## Logic according to dataloader

def normalize_str(txt) -> str:
    valid_chars = (" ", "'", "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z")
    
    new_txt = unidecode.unidecode(txt.lower().strip())
    
    res_arr = []
    for c in new_txt:
        if c in valid_chars:
            res_arr.append(c)
        else:
            res_arr.append(' ')
    res = ''.join(res_arr).strip()    
    return ' '.join(res.split())

In [11]:
data_dir = '../../'
datasets_dir = '../../datasets/'

In [12]:
# --- Building Manifest Files For RUSLAN --- #

# Function to build a manifest
def build_manifest(transcripts_path, manifest_path, wav_path, test_size=0.1, random_state=42):
    with open(transcripts_path, 'r') as fin:
        metadata = pd.read_csv(fin, sep='|', names=['file_name', 'transcript', 'normalized'])
        metadata_train, metadata_test = train_test_split(metadata, test_size=test_size, random_state=random_state)
        for m, m_path in [(metadata_train, 'train_ruslan.json'), (metadata_test, 'test_ruslan.json')]:
            with open(manifest_path + m_path, 'w', encoding='utf8') as fout:
                for row in tqdm(m.iterrows()):
                    file_id = row[1]['file_name']  # e.g. "cen4-fash-b"
                    audio_path = os.path.join(
                        manifest_path, wav_path, file_id + '.wav')

                    duration = librosa.core.get_duration(filename=audio_path)
                    
                    if duration > 12:
                        continue
                    
                    transcript = row[1]['transcript']

                    # Write the metadata to the manifest
                    metadata = {
                        "audio_filepath": audio_path,
                        "duration": duration,
                        "text": normalize_str(transcript)
                    }
                    json.dump(metadata, fout, ensure_ascii=False)
                    fout.write('\n')

# Building Manifests
print("******")
train_transcripts = datasets_dir + '/RUSLAN/metadata_RUSLAN_22200.csv'
manifest = datasets_dir + '/RUSLAN/'
build_manifest(train_transcripts, manifest, 'RUSLAN')
print("Manifests created.")

******


19980it [02:58, 111.65it/s]
2220it [00:18, 122.13it/s]

Manifests created.


In [6]:
# --- Building Manifest Files For MCV --- #
def tsv_to_manifest(tsv_files, manifest_file, prefix):
  manifests = []
  speakers = []
  for tsv_file in tsv_files:
    print('Processing: {0}'.format(tsv_file))
    dt = pd.read_csv(tsv_file, sep='\t', encoding='utf8')
    for index, row in dt.iterrows():
      try:
        entry = {}
        os.system("mkdir -p wavs/{0}".format(prefix))
        mp3_file = "clips/" + row['path'] # + ".mp3"
        wav_file = "wavs/{0}/".format(prefix) + row['path'] + ".wav"
        subprocess.check_output("sox {0} -c 1 -r 22050 {1}".format(mp3_file, wav_file), shell=True)
        duration = subprocess.check_output(
          "soxi -D {0}".format(wav_file), shell=True)
        entry['audio_filepath'] = wav_file
        entry['duration'] = float(duration)
        entry['text'] = normalize_str(row['sentence'])
        entry['speaker'] = row['client_id'][:10]
        speakers.append(row['client_id'])
        manifests.append(entry)
      except:
        print("SOMETHING WENT WRONG - IGNORING ENTRY")

  with codecs.open(manifest_file, 'w', encoding='utf-8') as fout:
    for m in manifests:
      fout.write(json.dumps(m, ensure_ascii=False) + '\n')
  print(len(set(speakers)))
  print('Done!')

tsv_to_manifest(['train.tsv'], 'train.json', 'train')
tsv_to_manifest(['test.tsv'], 'test.json', 'test')